In [2]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
import numpy as np

In [23]:
max_len = 60
step = 3
sentences = []
next_chars = []

DATA_DIR = "../data/"

In [24]:
text = ""
for file in os.listdir(DATA_DIR):
    if file.endswith(".txt"):
        text += open(os.path.join(DATA_DIR, file)).read().lower()

- Extrair sentenças de 3 caracteres
- Criar nova sentença a cada três caracteres
- Obter a sentença extraída
- Lista de caracteres únicos
- Dicionário que mapeia os caracteres unícos em indices
- One-hot encode

In [25]:
for i in range(0, len(text) - max_len, step):
    sentences.append(text[i: i + max_len])
    next_chars.append(text[i + max_len])
    
print('Sequence length: ', len(sentences))

Sequence length:  190435


In [26]:
chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Unique characters: ', len(chars))

Unique characters:  72


In [27]:
x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [28]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Criar a network

Utilizaremos um modelo com uma única camada LSTM 

In [36]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

Utilizando one-hot encode, utilizaremos a funcao de perca **categorical_crossentropy**

In [37]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [38]:
def sample(preds, temp=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [39]:
import random
import sys
model.fit(x, y, batch_size=128, epochs=30)
model.save('../models/model-1.h5')

Epoch 1/30
190435/190435 [==============================] - 153s 805us/step - loss: nan
Epoch 2/30
190435/190435 [==============================] - 153s 804us/step - loss: nan
Epoch 3/30
190435/190435 [==============================] - 153s 804us/step - loss: nan
Epoch 4/30
190435/190435 [==============================] - 153s 804us/step - loss: nan
Epoch 5/30
190435/190435 [==============================] - 2686s 14ms/step - loss: nan
Epoch 6/30
190435/190435 [==============================] - 2612s 14ms/step - loss: nan
Epoch 7/30
190435/190435 [==============================] - 153s 803us/step - loss: nan
Epoch 8/30
190435/190435 [==============================] - 153s 802us/step - loss: nan
Epoch 9/30
190435/190435 [==============================] - 153s 802us/step - loss: nan
Epoch 10/30
190435/190435 [==============================] - 153s 802us/step - loss: nan
Epoch 11/30
190435/190435 [==============================] - 153s 801us/step - loss: nan
Epoch 12/30
190435/190435 [===